In [1]:
!wget https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/train.zip
!wget https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/test.zip
!unzip train.zip
!unzip test.zip

--2025-11-27 16:30:47--  https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19229940 (18M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  18.34M  --.-KB/s    in 0.05s   

2025-11-27 16:30:47 (339 MB/s) - ‘train.zip’ saved [19229940/19229940]

--2025-11-27 16:30:47--  https://raw.githubusercontent.com/soydz/ia-saberpro-prediction/main/data/test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7966338 (7.6M) 

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import unicodedata
import re


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print('train', train.shape)
print('test', test.shape)

train (692500, 21)
test (296786, 20)


#### Funciones

In [12]:
def clean_data_base_2(data):
  data.drop(columns=['F_TIENEINTERNET.1', 'ID', 'PERIODO_ACADEMICO', 'E_PRIVADO_LIBERTAD'], inplace=True)

  data['E_PRGM_ACADEMICO'] = data['E_PRGM_ACADEMICO'].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode('ASCII'))
  data['E_PRGM_ACADEMICO'] = data['E_PRGM_ACADEMICO'].str.replace(r'\s+', ' ', regex=True).str.strip()

  reemplazos = {
    r'\bADMINISTRACIN\b': 'ADMINISTRACION',
    r'\bLOGSTICA\b': 'LOGISTICA',
    r'\bPBLICA\b': 'PUBLICA',
    r'\bGESTIN\b': 'GESTION',
    r'\bQUMICA\b': 'QUIMICA',
    r'\bFARMACUTICA\b': 'FARMACEUTICA',
    r'\bGASTRONOMA\b': 'GASTRONOMIA',
    r'\bPSICOSOCIALES\b': 'PSICOSOCIALES',
  }
  data['E_PRGM_ACADEMICO']= data['E_PRGM_ACADEMICO'].replace(reemplazos, regex=True)

  reemplazos = {
    'MERCADEO':'ECONOMIA',
    'MERCADEO NACIONAL E INTERNACIONAL':'ECONOMIA',
    'MERCADEO Y VENTAS':'ECONOMIA',
    'PROFESIONAL EN MERCADEO':'ECONOMIA',
    'PROFESIONAL EN MERCADEO EMPRESARIAL':'ECONOMIA'
  }
  data['E_PRGM_ACADEMICO']= data['E_PRGM_ACADEMICO'].replace(reemplazos)

  return data

Se agrupan los programas academicos segun la clasificación del SNIES.
El SNIES es el Sistema Nacional de Información de la Educación Superior de Colombia.

Definición de áreas SNIES (agrupadas en 6 grupos)

In [13]:
snies_areas = {
  'SALUD': [
    r'\b(microbiolog[iaí]|bioan[áa]lisis|gerontolog[iaí]|'
    r'seguridad\s+y\s+salud\s+(en\s+el\s+trabajo|para\s+el\s+trabajo))\b',
    r'\b(medicina|enfermer[iaí]|odontolog[iaí]|fisioterapia|nutrici[óo]n|'
    r'optometr[iaí]|bacteriolog[iaí]|fonoaudiolog[iaí]|psicolog[iaí]\s+cl[ií]nica|'
    r'terapia\s+(ocupacional|cardiorrespiratoria|respiratoria)|instrumentaci[óo]n\s+quir[uú]rgica|'
    r'ciencias\s+de\s+la\s+salud|salud\s+p[úu]blica|salud\s+ocupacional|'
    r'bioquimica\s+cl[ií]nica|microbiolog[iaí]\s+y\s+bioan[áa]lisis|'
    r'farmacia|gerontolog[iaí]|seguridad\s+y\s+salud\s+en\s+el\s+trabajo)\b',
    r'\bBACTERIOLOGIA\b',
    r'\bFONOAUDIOLOGIA\b',
    r'\bODONTOLOGIA\b',
    r'\bOPTOMETRIA\b',
    r'\bENFERMERIA\b',
    r'\bMICROBIOLOG[ÍI]A\b',
    r'\bGERONTOLOG[ÍI]A\b',
    r'TERAPIAS PSICOSOCIALES'
  ],

  'INGENIERIA': [
    r'\bINGENIERIA\s+(ADMINISTRATIVA|AERONAUTICA|COMERCIAL|DE\s+MERCADOS|'
    r'DE\s+PETROLEO\s+Y\s+GAS|DE\s+PROCESOS(\s+INDUSTRIALES)?|'
    r'DE\s+PRODUCCION(\s+ACUICOLA)?|DE\s+PRODUCTIVIDAD\s+Y\s+CALIDAD|'
    r'DE\s+TELECOMUNICACIONES|EN\s+(ENERG[IAÍ]|HIGIENE\s+Y\s+SEGURIDAD\s+OCUPACIONAL|'
    r'MULTIMEDIA|PROCESOS\s+INDUSTRIALES|PRODUCCION\s+ACUICOLA|'
    r'SEGURIDAD\s+Y\s+SALUD\s+PARA\s+EL\s+TRABAJO|SOFTWARE|TELECOMUNICACIONES)|'
    r'MULTIMEDIA|TOPOGRAFICA)\b',
    r'\b(SISTEMAS|COMPUTACION|INFORMATICA|TELEMATICA|'
    r'TELECOMUNICACIONES|ELECTRONICA|ELECTRICA|INDUSTRIAL|'
    r'MECANICA|QUIMICA|AMBIENTAL|CIVIL|BIOMEDICA|SOFTWARE|'
    r'GEOLOGICA|METALURGICA|ALIMENTOS|AGRONOMICA|'
    r'MINAS|PETROLEOS|AGRICOLA|AGROECOLOGICA|AGROFORESTAL|'
    r'AGROINDUSTRIAL|BIOLOGICA|BIOQUIMICA|BIOTECNOLOGICA|'
    r'DISENO\s+DE\s+PRODUCTO|MATERIALES|SONIDO|TRANSPORTE|'
    r'ELECTROMECANICA|MECATRONICA|NANOTECNOLOGIA|TOPOGRAFIA|FORESTAL|'
    r'NAVAL|PECUARIA|PESQUERA|SANITARIA|FINANCIERA|'
    r'CONTROL|AUTOMATIZACION|INSTRUMENTACION)\b',
    r'\b(ARQUITECTURA|CONSTRUCCION|CONSTRUCCIONES\s+CIVILES)\b',
    r'\bINGENIERA|INGENIERIA\b',
    r'URBANISMO',
  ],

  'SOCIALES': [
    r'\b(derecho|jurisprudencia|psicolog[iaí](?!\s+(cl[ií]nica|ocupacional))|'
    r'comunicaci[óo]n|periodismo|relaciones\s+(internacionales|econ[óo]micas\s+internacionales)|'
    r'ciencia\s+pol[ií]tica|trabajo\s+social|antropolog[iaí]|sociolog[iaí]|'
    r'gobierno\s+y\s+asuntos\s+p[úu]blicos|planeaci[óo]n\s+y\s+desarrollo\s+social|'
    r'estudios\s+(pol[ií]ticos|literarios|musicales|sociculturales)|'
    r'criminolog[iaí]|criminal[íi]stica|investigaci[óo]n\s+criminal|'
    r'ling[uü]istica|lenguajes)\b',
    r'\bANTROPOLOGIA\b',
    r'\bSOCIOLOGIA\b',
    r'\bCIENCIAS POLITICAS\b',
    r'\bCOMUNICACIONES\b',
    r'\bLINGUISTICA\b',
    r'\bLENGUAJES Y ESTUDIOS SOCIOCULTURALES\b',
    r'\bPSICOLOGIA\b',
    r'\bPROFESIONAL EN PSICOLOGIA\b',
    r'ARQUEOLOGIA',
    r'DESARROLLO TERRITORIAL',
    r'\b(GESTION\s+CULTURAL|GESTION\s+DEPORTIVA|GESTION\s+Y\s+DESARROLLO\s+URBANOS)\b',
    r'PEDAGOGIA'
  ],

  'EDUCACION': [
    r'\b(licenciatura|pedagog[iaí]|educaci[óo]n|docencia|enseñanza|'
    r'did[áa]ctica|formaci[óo]n\s+docente|desarrollo\s+familiar)\b',
  ],

  'ECONOMIA': [
    r'MERCADEO',
    r'\b(?:MERCADEO(?: NACIONAL E INTERNACIONAL| EMPRESARIAL)?|PROFESIONAL EN MERCADEO(?: EMPRESARIAL)?)\b',
    r'\b(?:MERCADEO|PROFESIONAL\s+EN\s+MERCADEO)(?:\s+(?:NACIONAL\s+E\s+INTERNACIONAL|EMPRESARIAL))?\b',
    r'\b(administraci[óo]n|contadur[iaí]|econom[iaí]|finanzas|mercado[lt]og[íi]a|'
    r'marketing|negocios|gerencia|log[ií]stica|turismo|gesti[óo]n\s+(empresarial|'
    r'cultural\s+y\s+comunicativa|del\s+dise[ñn]o|de\s+la\s+seguridad|'
    r'de\s+mercados|financiera|urbana|territorial)|comercio\s+(exterior|internacional)|'
    r'administraci[óo]n\s+tur[íi]stica\s+y\s+hotelera|'
    r'contadur[iaí]\s+(publica|internacional)|mercadeo|econom[iaí]a)\b',
    r'\b(?:MERCADEO|MARKETING|PUBLICIDAD)(?:\s+(?:INTERNACIONAL|NACIONAL|EMPRESARIAL|Y\s+(?:PUBLICIDAD|MERCADEO|VENTAS|INTERNACIONAL)|E\s+INTERNACIONAL))?\b',
    r'\bPROFESIONAL\s+EN\s+(?:MERCADEO|MARKETING)(?:\s+(?:EMPRESARIAL|Y\s+PUBLICIDAD))?\b',
    r'\b(?:ADMINISTRACI[ÓO]N|CONTADUR[ÍI]A|ECONOM[ÍI]A|FINANZAS|'
    r'GERENCIA|LOG[ÍI]STICA|TURISMO|NEGOCIOS|COMERCIO)\b',
  ],

  'CIENCIAS': [
    r'\b(matem[áa]tica|f[íi]sica|qu[íi]mica|biolog[iaí]|geolog[iaí]|'
    r'estad[ií]stica|ciencias\s+b[áa]sicas|ecolog[iaí]|'
    r'ciencias\s+(ambientales|biol[óo]gicas|del\s+mar|n[áa]uticas|'
    r'militares\s+(aeron[áa]uticas|navales)?)|geograf[iaí]|'
    r'astrof[íi]sica|geociencias|bioingenier[iaí]a|'
    r'biotecnolog[iaí]a|bioquimica|deporte|recreaci[óo]n|'
    r'entrenamiento\s+deportivo)\b',
    r'\bBIOLOGIA\b',
    r'\bECOLOGIA\b',
    r'\bMATEMATICAS\b',
    r'\bGEOGRAFIA\b',
    r'\bGEOLOGIA\b',
    r'\bDEPORTE\b',
    r'\bRECREACION\b',
    r'\bENTRENAMIENTO DEPORTIVO\b',
    r'\bPROFESIONAL EN DEPORTE\b',
    r'\bPROFESIONAL EN ENTRENAMIENTO DEPORTIVO\b',
    r'\bCIENCIAS DEL DEPORTE\b',
    r'\bCIENCIAS DEL DEPORTE Y LA RECREACION\b',
    r'\bCIENCIAS MILITARES\b',
    r'\bCIENCIAS NAVALES PARA OFICIALES DE INFANTERIA DE MARINA\b',
    r'\bCIENCIAS NAVALES PARA OFICIALES NAVALES\b',
    r'ASTRONOMIA',
    r'OCEANOGRAFIA'
  ],

  'ARTES_HUMANIDADES': [
    r'\b(filosof[iaí]|historia|literatura|lenguas|traducci[óo]n|'
    r'arte|danza|dram[áa]tico|cine|audiovisuales|televisi[óo]n|'
    r'radio|producci[óo]n\s+(cinematogr[áa]fica|audiovisual|musical)|'
    r'dise[ñn]o\s+(gr[áa]fico|industrial|interior|de\s+modas|'
    r'de\s+espacios|digital|interactivo|visual|multimedia|'
    r'de\s+vestuario|escenogr[áa]fico)|gastronom[iaí]a|'
    r'culinaria|teolog[iaí]|musicolog[iaí]|interpretaci[óo]n\s+musical|'
    r'fotograf[iaí]a|bibliotecolog[iaí]a|archiv[íi]stica|'
    r'creaci[óo]n\s+literaria|espanol|filologia|'
    r'musica|formaci[óo]n\s+musical|estudios\s+(literarios|musicales|culturales))\b',
    r'\bARTES\b',
    r'\bMUSICA\b',
    r'\bFILOSOFIA\b',
    r'\bFILOSOFIA Y HUMANIDADES\b',
    r'\bFILOSOFIA Y LETRAS\b',
    r'\bESPANOL Y FILOLOGIA CLASICA\b',
    r'\bFILOLOGIA E IDIOMAS\b',
    r'\bFILOLOGIA HISPANICA\b',
    r'\bCIENCIAS BIBLICAS\b',
    r'\bTEOLOGIA\b',
    r'\bDISENO\b',
    r'\bDISENO DE MEDIOS INTERACTIVOS\b',
    r'\bDISENO Y GESTION DE LA MODA\b',
    r'\bDISENO Y PRODUCCION DE MODA\b',
    r'\bCREACION LITERARIA\b',
    r'\bESTUDIOS Y GESTION CULTURAL\b',
    r'\bCIENCIAS\s+DE\s+LA\s+INFORMACION\s+Y\s+LA\s+DOCUMENTACION\b',
    r'ANIMACION',
    r'TEATRO',
    r'DIRECCION DE BANDA',
    r'CONSERVACION Y RESTAURACION DE BIENES MUEBLES',
    r'CROSSMEDIA',
    r'NARRATIVAS DIGITALES',
  ],

  'AGROPECUARIO': [
    r'\b(agronom[iaí]|agroecolog[iaí]a|agroforestal|agroindustrial|'
    r'agropecuaria|zootecnia|acuicultura|pesquera|'
    r'producci[óo]n\s+agroindustrial|agronegocios|'
    r'profesional\s+en\s+(agroindustria|agronegocios))\b',
    r'\bAGRONOMIA\b',
    r'\bAGRONOMIA DEL TROPICO HUMEDO\b',
  ]
}


# Función para mapear programa a área
def mapper_prgm_academico_to_snies(programa):
  if not isinstance(programa, str):
      return 'OTRO'

  # Corrección de errores tipográficos frecuentes
  p = programa.upper().strip()
  p = re.sub(r'[^\w\s]', ' ', p)  # Elimina puntuación
  p = re.sub(r'\s+', ' ', p)      # Elimina espacios múltiples

  # Corección de errores de digitación del dataset
  p = re.sub(r'\bADMINSITRACION\b', 'ADMINISTRACION', p)
  p = re.sub(r'\bINTRUMENTACION\b', 'INSTRUMENTACION', p)
  p = re.sub(r'\bCOMUNICACIN\b', 'COMUNICACION', p)
  p = re.sub(r'\bMERCADEO\b', 'MERCADOTECNIA', p)
  p = re.sub(r'\bCOMUNICACIN\b', 'COMUNICACION', p)
  p = re.sub(r'\bPSICOLOGIA\b', 'PSICOLOGIA', p)

  # Busca en cada área
  for area, patrones in snies_areas.items():
      for patron in patrones:
          if re.search(patron, p, re.IGNORECASE):
              return area
  return 'OTRO'

In [14]:
def clean_prgm_departamento_to_region(data):
  region_map = {
    # Caribe
    'ATLANTICO': 'CARIBE', 'BOLIVAR': 'CARIBE', 'SUCRE': 'CARIBE', 'MAGDALENA': 'CARIBE',
    'LA GUAJIRA': 'CARIBE', 'CESAR': 'CARIBE', 'SAN ANDRES': 'CARIBE',

    # Andina
    'BOGOTÁ': 'ANDINA', 'ANTIOQUIA': 'ANDINA', 'HUILA': 'ANDINA', 'CUNDINAMARCA': 'ANDINA',
    'TOLIMA': 'ANDINA', 'VALLE': 'ANDINA', 'QUINDIO': 'ANDINA', 'RISARALDA': 'ANDINA',
    'CORDOBA': 'ANDINA', 'BOYACA': 'ANDINA', 'NARIÑO': 'ANDINA', 'CALDAS': 'ANDINA',
    'SANTANDER': 'ANDINA', 'NORTE SANTANDER': 'ANDINA',

    # Pacífica
    'CAUCA': 'PACIFICA', 'CHOCO': 'PACIFICA', 'NARIÑO': 'PACIFICA',

    # Orinoquía
    'META': 'ORINOQUIA', 'CASANARE': 'ORINOQUIA', 'ARAUCA': 'ORINOQUIA', 'GUAVIARE': 'ORINOQUIA',

    # Amazónica
    'AMAZONAS': 'AMAZONICA', 'CAQUETA': 'AMAZONICA', 'PUTUMAYO': 'AMAZONICA',
    'VAUPES': 'AMAZONICA'
  }

  data['E_PRGM_DEPARTAMENTO'] = data['E_PRGM_DEPARTAMENTO'].map(region_map).fillna('OTRO')

  return data

In [15]:
def clean_nulos_2(data):
  imputer = SimpleImputer(strategy='most_frequent')
  data['F_ESTRATOVIVIENDA'] = imputer.fit_transform(data[['F_ESTRATOVIVIENDA']])[:, 0]
  data['F_TIENEINTERNET'] = imputer.fit_transform(data[['F_TIENEINTERNET']])[:, 0]
  data['F_TIENECOMPUTADOR'] = imputer.fit_transform(data[['F_TIENECOMPUTADOR']])[:, 0]
  data['F_TIENELAVADORA'] = imputer.fit_transform(data[['F_TIENELAVADORA']])[:, 0]
  data['F_TIENEAUTOMOVIL'] = imputer.fit_transform(data[['F_TIENEAUTOMOVIL']])[:, 0]

  data['E_VALORMATRICULAUNIVERSIDAD'] = imputer.fit_transform(data[['E_VALORMATRICULAUNIVERSIDAD']])[:, 0]
  data['E_PAGOMATRICULAPROPIO'] = imputer.fit_transform(data[['E_PAGOMATRICULAPROPIO']])[:, 0]

  data['F_EDUCACIONPADRE'] = data.groupby('F_ESTRATOVIVIENDA')['F_EDUCACIONPADRE'].transform(lambda x: x.fillna(x.mode()[0]))
  data['F_EDUCACIONMADRE'] = data.groupby('F_ESTRATOVIVIENDA')['F_EDUCACIONMADRE'].transform(lambda x: x.fillna(x.mode()[0]))

  data['E_HORASSEMANATRABAJA'] = data['E_HORASSEMANATRABAJA'].map({'0': 0, 'Menos de 10 horas': 9, 'Entre 11 y 20 horas': 15.5, 'Entre 21 y 30 horas': 25.5, 'Más de 30 horas': 31})
  data['E_HORASSEMANATRABAJA'] = data['E_HORASSEMANATRABAJA'].fillna((data['E_HORASSEMANATRABAJA'].mean()).round(3))

  return data

In [16]:
def categorical_to_numeric(data):
  data = pd.get_dummies(data, columns=['E_VALORMATRICULAUNIVERSIDAD'], dtype=np.uint8)
  data.drop(columns=['E_VALORMATRICULAUNIVERSIDAD_No pagó matrícula'], inplace=True)

  data = pd.get_dummies(data, columns=['F_ESTRATOVIVIENDA'], dtype=np.uint8)
  data.drop(columns=['F_ESTRATOVIVIENDA_Sin Estrato'], inplace=True)
  data.columns = data.columns.str.replace('_Estrato ', '_')

  data['F_TIENEINTERNET'] = data['F_TIENEINTERNET'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENELAVADORA'] = data['F_TIENELAVADORA'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENEAUTOMOVIL'] = data['F_TIENEAUTOMOVIL'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['E_PAGOMATRICULAPROPIO'] = data['E_PAGOMATRICULAPROPIO'].map({'Si': 1, 'No': 0}).astype(np.int8)
  data['F_TIENECOMPUTADOR'] = data['F_TIENECOMPUTADOR'].map({'Si': 1, 'No': 0}).astype(np.int8)

  mapping = {
    'Ninguno': 0,
    'No sabe': 0,
    'No Aplica': 0,
    'Primaria incompleta': 1,
    'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3,
    'Secundaria (Bachillerato) completa': 5,
    'Técnica o tecnológica incompleta': 7,
    'Técnica o tecnológica completa': 10,
    'Educación profesional incompleta': 13,
    'Educación profesional completa': 20,
    'Postgrado': 40
  }

  data['F_EDUCACIONMADRE'] = data['F_EDUCACIONMADRE'].map(mapping).astype(np.uint8)
  data['F_EDUCACIONPADRE'] = data['F_EDUCACIONPADRE'].map(mapping).astype(np.uint8)

  data = pd.get_dummies(data, columns=['E_PRGM_DEPARTAMENTO'], prefix='REGION', dtype=np.uint8)

  data = pd.get_dummies(data, columns=['AREA_SNIES'], dtype=np.uint8)

  return data

In [17]:
def normalize_data(data):
  data['E_HORASSEMANATRABAJA'] = (data['E_HORASSEMANATRABAJA'] - data['E_HORASSEMANATRABAJA'].min()) / (data['E_HORASSEMANATRABAJA'].max() - data['E_HORASSEMANATRABAJA'].min())
  data['F_EDUCACIONMADRE'] = (data['F_EDUCACIONMADRE'] - data['F_EDUCACIONMADRE'].min()) / (data['F_EDUCACIONMADRE'].max() - data['F_EDUCACIONMADRE'].min())
  data['F_EDUCACIONPADRE'] = (data['F_EDUCACIONPADRE'] - data['F_EDUCACIONPADRE'].min()) / (data['F_EDUCACIONPADRE'].max() - data['F_EDUCACIONPADRE'].min())

  return data

En el dataset, las columnas nombradas como `INDICADOR_1`, `INDICADOR_2`, `INDICADOR_3` e `INDICADOR_4`, no queda muy claro que representan, que información brindan. Se procede a elimar estos datos para comprobar si aportan al modelo o son contraproducentes.

Despues de evaluar el modelo con estos datos, se realiza una segunda evaluacción del modelo, esta vez sin estas columnas presentes, esperando observar un cambio en el Test Accuracy. De esta forma, tomar la decisión de conservarlas o eliminarlas.

In [18]:
def delete_indicadors(data):
  data.drop(['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'], axis=1, inplace=True)

Se crea una columna, para intentar mejorar el valor predictivo del modelo.

Se parte de la premisa de que el esfuerzo de un estudiante al pagar su matricula, cambia si es estrato 1 o estrato 6.

In [19]:
def new_column_ESFUERZO_RELATIVO(data):
  PAGOMATRICULAPROPIO = data['E_PAGOMATRICULAPROPIO'].map({
    'Si': 1,
    'No': 0
  }).astype(np.int8)

  estrato_map = {
      'Estrato 1': 1,
      'Estrato 2': 2,
      'Estrato 3': 3,
      'Estrato 4': 4,
      'Estrato 5': 5,
      'Estrato 6': 6,
      'Sin Estrato': 3.5
  }

  ESTRATOVIVIENDA = data['F_ESTRATOVIVIENDA'].map(estrato_map).astype(np.int8)

  data['ESFUERZO_RELATIVO'] = PAGOMATRICULAPROPIO / ESTRATOVIVIENDA + 1e-6

In [20]:
def clean_saber_pro_5(data):
  data = clean_data_base_2(data)
  data = clean_nulos_2(data)
  delete_indicadors(data)
  new_column_ESFUERZO_RELATIVO(data)

  return data

#### Main

Limpieza de los datos

In [21]:
df = train.copy()
df = clean_saber_pro_5(df)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   E_PRGM_ACADEMICO             692500 non-null  object 
 1   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 2   E_VALORMATRICULAUNIVERSIDAD  692500 non-null  object 
 3   E_HORASSEMANATRABAJA         692500 non-null  float64
 4   F_ESTRATOVIVIENDA            692500 non-null  object 
 5   F_TIENEINTERNET              692500 non-null  object 
 6   F_EDUCACIONPADRE             692500 non-null  object 
 7   F_TIENELAVADORA              692500 non-null  object 
 8   F_TIENEAUTOMOVIL             692500 non-null  object 
 9   E_PAGOMATRICULAPROPIO        692500 non-null  object 
 10  F_TIENECOMPUTADOR            692500 non-null  object 
 11  F_EDUCACIONMADRE             692500 non-null  object 
 12  RENDIMIENTO_GLOBAL           692500 non-null  object 
 13 

Pasa la variable a predecir, de categorica a numérica

In [23]:
mapping = {
    'alto': 4,
    'medio-alto': 3,
    'medio-bajo': 2,
    'bajo': 1
}

df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapping).astype('int8')

### Test

**LightGBM** (Light Gradient Boosting Machine) es una biblioteca de **aprendizaje automático** optimizada para tareas de clasificación y regresión. Se basa en el algoritmo de **Gradient Boosting Decision Trees** (GBDT), pero con mejoras clave que lo hacen más rápido y eficiente, especialmente en grandes volúmenes de datos. LightGBM utiliza técnicas como **histogramas** para acelerar la construcción de árboles, soporte nativo para variables categóricas, y optimización en la paralelización del entrenamiento. Esto lo convierte en una opción popular para problemas de **gran escala** y **alto rendimiento**.

In [24]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split

Separa del dataset la variable a predecir

In [25]:
X = df.drop(columns=['RENDIMIENTO_GLOBAL'])
y = df.RENDIMIENTO_GLOBAL

Convierte las columnas a categoría, para aprovechar las optimizaciones de LightGBM

In [26]:
for col in X.columns:
    X[col] = X[col].astype('category')

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   E_PRGM_ACADEMICO             692500 non-null  category
 1   E_PRGM_DEPARTAMENTO          692500 non-null  category
 2   E_VALORMATRICULAUNIVERSIDAD  692500 non-null  category
 3   E_HORASSEMANATRABAJA         692500 non-null  category
 4   F_ESTRATOVIVIENDA            692500 non-null  category
 5   F_TIENEINTERNET              692500 non-null  category
 6   F_EDUCACIONPADRE             692500 non-null  category
 7   F_TIENELAVADORA              692500 non-null  category
 8   F_TIENEAUTOMOVIL             692500 non-null  category
 9   E_PAGOMATRICULAPROPIO        692500 non-null  category
 10  F_TIENECOMPUTADOR            692500 non-null  category
 11  F_EDUCACIONMADRE             692500 non-null  category
 12  ESFUERZO_RELATIVO            692500 non-null

Separa una parte de los datos para testear el modelo

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y - 1,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f"Train: {X_train.shape} | Test: {X_test.shape}")
print(f"Distribución train: {np.bincount(y_train)/len(y_train)}")

Train: (554000, 13) | Test: (138500, 13)
Distribución train: [0.24980144 0.24877256 0.24782491 0.25360108]


Se configuran los parámetros del modelo para mejorar el rendimiento y evitar el sobreajuste.

- `objective`: `multiclass`: Indica que se está realizando una clasificación multiclase (es decir, más de dos clases). El modelo ajustará su objetivo para esto.

- `num_class`: `4`: Especifica que hay 4 clases en el problema de clasificación.

- `metric`: `multi_logloss`: Métrica utilizada para la evaluación del modelo. En este caso, se utiliza log loss para la clasificación multiclase. Un valor más bajo indica un mejor modelo.

- `boosting_type`: `gbdt`: El tipo de boosting utilizado es Gradient Boosting Decision Tree (GBDT), que es el enfoque por defecto en LightGBM.

- `num_leaves`: `95`: Especifica el número máximo de hojas por árbol. Un valor mayor implica árboles más complejos, con mayor capacidad para aprender patrones complejos, pero también puede aumentar el riesgo de sobreajuste (overfitting).

- `min_data_in_leaf`: `180`: El número mínimo de muestras que debe contener una hoja del árbol. Este parámetro ayuda a controlar el sobreajuste, al evitar que se creen hojas con muy pocos ejemplos.

- `min_sum_hessian_in_leaf`: `20`: Es un parámetro que mejora la estabilidad del modelo, especialmente en problemas con clases desbalanceadas. Controla el número mínimo de "hessianos" (una medida de curvatura) necesarios en una hoja.

- `cat_l2`: `15.0` y `cat_smooth`: `20.0`: Son parámetros de regularización específicos para las variables categóricas. Ayudan a evitar que el modelo se sobreajuste a categorías que tienen pocos ejemplos.

- `feature_fraction`: `0.75`: La fracción de características a considerar en cada iteración del entrenamiento. Un valor de 0.75 significa que LightGBM utilizará el 75% de las características disponibles en cada iteración para construir los árboles. Esto puede mejorar la generalización y reducir el sobreajuste.

- `bagging_fraction`: `0.8` y `bagging_freq`: `3`: Estas son técnicas de bagging (muestreo aleatorio de los datos):

- `bagging_fraction`: Especifica la fracción de los datos que se utilizan en cada iteración (en este caso el 80% de los datos).

- `bagging_freq`: Controla la frecuencia con la que se aplica el bagging. Con un valor de 3, LightGBM aplicará bagging cada 3 iteraciones.

- `learning_rate`: `0.012`: Tasa de aprendizaje. Un valor más bajo hace que el modelo aprenda más lentamente, lo cual generalmente mejora la estabilidad del modelo, pero requiere más boosting rounds (iteraciones).

- `lambda_l1`: `1.0` y `lambda_l2`: `1.5`: Son parámetros de regularización L1 y L2. Ayudan a evitar el sobreajuste penalizando grandes coeficientes en las características.

- `max_depth`: `10`: Indica que ningún árbol tendrá más de 10 niveles de profundidad.

- `verbosity`: `-1`: Controla el nivel de salida de los registros del modelo. `-1` suprime la salida, lo que significa que no verás detalles del proceso de entrenamiento.

- `random_state`: `42`: Fija la semilla aleatoria para asegurar la reproducibilidad de los resultados.

- `is_unbalance`: `True`: Activa el ajuste automático de pesos por clase para mejorar el desempeño en clases difíciles de predecir.


In [29]:
params = {
  'objective': 'multiclass',
  'num_class': 4,
  'metric': 'multi_logloss',
  'boosting_type': 'gbdt',
  'num_leaves': 95,
  'min_data_in_leaf': 180,
  'min_sum_hessian_in_leaf': 20,
  'cat_l2': 15.0,
  'cat_smooth': 20.0,
  'feature_fraction': 0.75,
  'bagging_fraction': 0.8,
  'bagging_freq': 3,
  'learning_rate': 0.012,
  'lambda_l1': 1.0,
  'lambda_l2': 1.5,
  'max_depth': 10,
  'verbosity': -1,
  'random_state': 42,
  'is_unbalance': True,
}

Se crean los conjuntos de datos para entrenamiento y validación, especificando las características categóricas.

In [30]:
train_data = lgb.Dataset(X_train, y_train, categorical_feature=X_train.columns.tolist())
val_data = lgb.Dataset(X_test, y_test, reference=train_data)

Se entrena el modelo con un número máximo de 2000 iteraciones, aplicando `early stopping` para detener el entrenamiento si no hay mejora durante 100 iteraciones consecutivas. Esto asegura que el modelo se entrene de manera eficiente y no se sobreajuste a los datos de entrenamiento.

El código también registra el progreso del entrenamiento cada 200 iteraciones para monitorear el desempeño del modelo.

In [31]:
model = lgb.train(
  params, train_data,
  valid_sets=[val_data],
  num_boost_round=2000,
  callbacks=[
    lgb.early_stopping(100),
    lgb.log_evaluation(200)
  ]
)

Training until validation scores don't improve for 100 rounds
[200]	valid_0's multi_logloss: 1.2123
[400]	valid_0's multi_logloss: 1.20073
[600]	valid_0's multi_logloss: 1.19799
[800]	valid_0's multi_logloss: 1.19671
[1000]	valid_0's multi_logloss: 1.19606
[1200]	valid_0's multi_logloss: 1.19565
[1400]	valid_0's multi_logloss: 1.19559
Early stopping, best iteration is:
[1301]	valid_0's multi_logloss: 1.19557


Evalua el rendimiento del modelo

In [32]:
y_pred = model.predict(X_test).argmax(axis=1) + 1
train_acc = accuracy_score(y_train + 1, model.predict(X_train).argmax(axis=1) + 1)
test_acc = accuracy_score(y_test + 1, y_pred)

print("\n📊 Resultados LightGBM:")
print(f"   Train Accuracy: {train_acc:.4f}")
print(f"   Test Accuracy:  {test_acc:.4f}")
print(f"   Overfitting:    {train_acc - test_acc:.4f} (ideal < 0.05)")


📊 Resultados LightGBM:
   Train Accuracy: 0.4781
   Test Accuracy:  0.4355
   Overfitting:    0.0426 (ideal < 0.05)


#### Resultados de Rendimiento, con diferentes preprocesamientos.

Modelo

```python
params = {
  'objective': 'multiclass',
  'num_class': 4,
  'metric': 'multi_logloss',
  'boosting_type': 'gbdt',
  'num_leaves': 127,          
  'min_data_in_leaf': 150,    
  'min_sum_hessian_in_leaf': 10,
  'cat_l2': 5.0,              
  'cat_smooth': 10.0,
  'feature_fraction': 0.85,
  'bagging_fraction': 0.9,
  'bagging_freq': 2,
  'learning_rate': 0.02,      
  'lambda_l1': 0.5,
  'lambda_l2': 0.5,
  'max_depth': -1,           
  'verbosity': -1,
  'random_state': 42
}

train_data = lgb.Dataset(X_train, y_train, categorical_feature=X_train.columns.tolist())
val_data = lgb.Dataset(X_test, y_test, reference=train_data)

model = lgb.train(
  params, train_data,
  valid_sets=[val_data],
  num_boost_round=2000,
  callbacks=[
    lgb.early_stopping(100),
    lgb.log_evaluation(200)
  ]
)
```

Se usan diferentes estrategias de preprocesado de los datos y se evalua, con la intencción de encontrar la que mejor resultados brinde.

Rendimiento del modelo con:
```python
  def clean_saber_pro_4(data):
    data = clean_data_base_2(data)
    data = clean_nulos_2(data)

    return data
```
    📊 Resultados LightGBM:
    Train Accuracy: 0.6218
    Test Accuracy:  0.4245
    Overfitting:    0.1972 (ideal < 0.05)

`Overfitting` muy alto

---

Rendimiento del modelo con:
```python
  def clean_saber_pro_4(data):
    data = clean_data_base_2(data)
    data['AREA_SNIES'] = data['E_PRGM_ACADEMICO'].apply(mapper_prgm_academico_to_snies)
    data.drop('E_PRGM_ACADEMICO', axis=1, inplace=True)
    data = clean_nulos_2(data)

    return data
```
    📊 Resultados LightGBM:
    Train Accuracy: 0.5896
    Test Accuracy:  0.4061
    Overfitting:    0.1836 (ideal < 0.05)

`Overfitting` muy alto

---

Rendimiento del modelo con:
```python
def clean_saber_pro_4(data):
  data = clean_data_base_2(data)
  data['AREA_SNIES'] = data['E_PRGM_ACADEMICO'].apply(mapper_prgm_academico_to_snies)
  data.drop('E_PRGM_ACADEMICO', axis=1, inplace=True)
  data = clean_nulos_2(data)
  delete_indicadors(data)

  return data
```

    📊 Resultados LightGBM:
    Train Accuracy: 0.4592
    Test Accuracy:  0.4194
    Overfitting:    0.0398 (ideal < 0.05)

`Overfitting` es aceptable

---

Rendimiento del modelo con:
```python
  def clean_saber_pro_4(data):
    data = clean_data_base_2(data)
    data = clean_nulos_2(data)
    delete_indicadors(data)

    return data
```
    📊 Resultados LightGBM:
    Train Accuracy: 0.4915
    Test Accuracy:  0.4358
    Overfitting:    0.0557 (ideal < 0.05)

---
Rendimiento del modelo con:
```python
  def clean_saber_pro_4(data):
    data = clean_data_base_2(data)
    data['AREA_SNIES'] = data['E_PRGM_ACADEMICO'].apply(mapper_prgm_academico_to_snies)
    data.drop('E_PRGM_ACADEMICO', axis=1, inplace=True)
    data = clean_prgm_departamento_to_region(data)
    data = clean_nulos_2(data)
    delete_indicadors(data)

    return data
```

    📊 Resultados LightGBM:
    Train Accuracy: 0.4332
    Test Accuracy:  0.4082
    Overfitting:    0.0250 (ideal < 0.05)

`Overfitting` mejora, pero el rendimiento en test en considerablemente menor que con otras estrategias.

Se ajustan los parametros:
```python
  params = {
    'objective': 'multiclass',
    'num_class': 4,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 95,
    'min_data_in_leaf': 180,
    'min_sum_hessian_in_leaf': 20,
    'cat_l2': 15.0,
    'cat_smooth': 20.0,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'learning_rate': 0.015,
    'lambda_l1': 1.0,
    'lambda_l2': 1.5,
    'max_depth': 10,
    'verbosity': -1,
    'random_state': 42,

    'is_unbalance': True,
}
```


Rendimiento del modelo con:
```python
  def clean_saber_pro_4(data):
    data = clean_data_base_2(data)
    data = clean_nulos_2(data)
    delete_indicadors(data)

    return data
```

    📊 Resultados LightGBM:
    Train Accuracy: 0.4827
    Test Accuracy:  0.4354
    Overfitting:    0.0473 (ideal < 0.05)


Se ajusta el parametro `learning_rate` del modelo

    'learning_rate': 0.012,

---

    📊 Resultados LightGBM:
    Train Accuracy: 0.4781
    Test Accuracy:  0.4355
    Overfitting:    0.0426 (ideal < 0.05)

Se consigue un buen valor de test, con un Overfitting aceptable.

### Preparacion archivo Kaggle

Entrenamiento con todo el dataset

In [33]:
full_data = lgb.Dataset(X, y - 1, categorical_feature=X.columns.tolist())
model_final = lgb.train(
    params, full_data,
    num_boost_round=model.best_iteration
)

Se guardan los ids de test, para al final usarlos en la creación del archivo para la submission.

En test se crea una columna que no se crea en train. Se elimina esta.

In [34]:
test = pd.read_csv('test.csv')
test_ids = test.ID
df_test = clean_saber_pro_5(test)

In [35]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   E_PRGM_ACADEMICO             296786 non-null  object 
 1   E_PRGM_DEPARTAMENTO          296786 non-null  object 
 2   E_VALORMATRICULAUNIVERSIDAD  296786 non-null  object 
 3   E_HORASSEMANATRABAJA         296786 non-null  float64
 4   F_ESTRATOVIVIENDA            296786 non-null  object 
 5   F_TIENEINTERNET              296786 non-null  object 
 6   F_EDUCACIONPADRE             296786 non-null  object 
 7   F_TIENELAVADORA              296786 non-null  object 
 8   F_TIENEAUTOMOVIL             296786 non-null  object 
 9   E_PAGOMATRICULAPROPIO        296786 non-null  object 
 10  F_TIENECOMPUTADOR            296786 non-null  object 
 11  F_EDUCACIONMADRE             296786 non-null  object 
 12  ESFUERZO_RELATIVO            296786 non-null  float64
dtyp

Convierte las columnas a categoría, al igual que se hizo con el dataset de entrenamiento.

In [36]:
df_test = df_test[X.columns]
for col in df_test.columns:
    df_test[col] = df_test[col].astype('category')

In [37]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   E_PRGM_ACADEMICO             296786 non-null  category
 1   E_PRGM_DEPARTAMENTO          296786 non-null  category
 2   E_VALORMATRICULAUNIVERSIDAD  296786 non-null  category
 3   E_HORASSEMANATRABAJA         296786 non-null  category
 4   F_ESTRATOVIVIENDA            296786 non-null  category
 5   F_TIENEINTERNET              296786 non-null  category
 6   F_EDUCACIONPADRE             296786 non-null  category
 7   F_TIENELAVADORA              296786 non-null  category
 8   F_TIENEAUTOMOVIL             296786 non-null  category
 9   E_PAGOMATRICULAPROPIO        296786 non-null  category
 10  F_TIENECOMPUTADOR            296786 non-null  category
 11  F_EDUCACIONMADRE             296786 non-null  category
 12  ESFUERZO_RELATIVO            296786 non-null

Predicción

In [38]:
y_pred_final = model_final.predict(df_test).argmax(axis=1) + 1

Pasa la variable predictora de numero a categorica

In [39]:
mapping = {
    1: "bajo",
    2: "medio-bajo",
    3: "medio-alto",
    4: "alto"
}

pred_str = pd.Series(y_pred_final).map(mapping).to_numpy()

In [40]:
pred_str

array(['bajo', 'medio-bajo', 'alto', ..., 'medio-alto', 'alto', 'alto'],
      dtype=object)

Generación de archivo

In [41]:
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': pred_str
})

submission.to_csv('saber_pro_2025-2_submission-99.csv', index=False)

In [42]:
!head saber_pro_2025-2_submission-99.csv

ID,RENDIMIENTO_GLOBAL
550236,bajo
98545,medio-bajo
499179,alto
782980,bajo
785185,bajo
58495,bajo
705444,alto
557548,alto
519909,bajo
